In [19]:
# Loading in packages
from PIL import Image
import numpy as np
import tensorflow as tf
import cv2
from tensorflow import keras
from moviepy.video.compositing.concatenate import concatenate_videoclips
import moviepy  
from moviepy.editor import *
import math

In [2]:
# Loading in video and variables
class Edit:
    def __init__(self,video_data_path):
        self.video_data_path = video_data_path
        self.vidcap = cv2.VideoCapture(self.video_data_path)
        self.frame_count = int(self.vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.current_frame = 0
        self.start_frame = None
        self.list_start_frame =[]
        self.list_end_frame =[]
        self.end_frame = None
        self.end_video =False
        self.fps = 60
        # Loading in model
        self.model = keras.models.load_model('model_22_01_11_notts_tuam')
    
    def read_frame(self,frame):
        self.vidcap.set(1,frame)
        ret, image = self.vidcap.read()
        imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        temp_image = Image.fromarray(imageRGB)
        # temp_image = Image.open("temp.jpg")
        temp_image = temp_image.resize((256, 256))
        temp_array = np.asarray(temp_image)
        output = np.empty((1,256,256,3))
        output[0] = temp_array
        return output

    def kayak_check(self,frame,start):
        test_frame = self.read_frame(frame)
        temp_predict = self.model.predict(test_frame)
        current_predict = temp_predict[0,1]
        if start:
            if current_predict > 10:
                return True, frame
            elif current_predict <10 and current_predict > 5:
                return False, (frame+1)
            else: 
                return False, (frame+60)
        else:
            if current_predict > 10:
                return False, (frame+60)
            elif current_predict <10 and current_predict > 5:
                return False, (frame+1)
            else: 
                return True, frame


In [20]:
path_to_vid = './Video_source/Tuam_video_3.MP4'
file_name = '22_01_12_02_full_test.mp4'
source_video_clip =  moviepy.VideoFileClip(path_to_vid)
subclip_dict_list = []

subclip_details_dict = {}

subclip_list = []

my_edit = Edit(path_to_vid)

while not my_edit.end_video:
    while my_edit.start_frame == None and  my_edit.end_video == False :
        if my_edit.current_frame >= my_edit.frame_count:
            my_edit.end_video = True
        else:
            start, my_edit.current_frame = my_edit.kayak_check(my_edit.current_frame,True)
            if start:
                my_edit.start_frame = my_edit.current_frame
                subclip_details_dict["start time"] = my_edit.start_frame/my_edit.fps
                my_edit.current_frame += 30
    # print(my_edit.start_frame)
    count = 0
    while my_edit.end_frame == None and  my_edit.end_video == False :
        if my_edit.current_frame >= my_edit.frame_count:
            my_edit.end_frame =my_edit.frame_count-1
            subclip_details_dict["end time"] = my_edit.end_frame/my_edit.fps
            my_edit.end_video = True
            break
        else:
            end, my_edit.current_frame = my_edit.kayak_check(my_edit.current_frame,False)
            if end:
                count += 1
                if count > 3:
                    my_edit.end_frame = my_edit.current_frame
                    subclip_details_dict["end time"] = my_edit.end_frame/my_edit.fps
                    end = False 
                my_edit.current_frame += 20
        
    # print(my_edit.end_frame)
   
    if my_edit.start_frame != None and my_edit.end_frame != None:
        my_edit.list_start_frame.append(my_edit.start_frame)
        my_edit.list_end_frame.append(my_edit.end_frame)
        start_time =  str(math.floor(subclip_details_dict["start time"]/60))+ ':' + str(round(((subclip_details_dict["start time"]) % 60),2))
        
        end_time = str(math.floor(subclip_details_dict["end time"]/60))+ ':' + str(round(((subclip_details_dict["end time"]) % 60),2))
        print('Start_time = '+start_time +'    End_time = ' + end_time)
        my_edit.start_frame = None 
        my_edit.end_frame = None
        subclip_dict_list.append(subclip_details_dict.copy())
    else:
        my_edit.end_video = True

for subclip in subclip_dict_list:
    subclip_source_video = source_video_clip
    current_subclip = subclip_source_video.subclip(subclip["start time"], subclip["end time"])
    subclip_list.append(current_subclip)
final_clip = concatenate_videoclips(subclip_list)

# source_video_clip.subclip(subclip["start time"], subclip["end time"])
final_clip.write_videofile(file_name)

Start time:  0:0.0
End time:  0:4.5
Start_time = 0:0.0    End_time = 0:4.5
Start time:  0:23.83
End time:  1:0.4
Start_time = 0:23.83    End_time = 1:0.4
Start time:  1:6.73
End time:  1:12.23
Start_time = 1:6.73    End_time = 1:12.23
Start time:  1:13.78
End time:  1:22.7
Start_time = 1:13.78    End_time = 1:22.7
Start time:  1:26.37
End time:  1:32.33
Start_time = 1:26.37    End_time = 1:32.33
Start time:  1:39.67
End time:  2:11.32
Start_time = 1:39.67    End_time = 2:11.32
Start time:  2:11.78
End time:  2:15.28
Start_time = 2:11.78    End_time = 2:15.28
Start time:  2:33.62
End time:  2:54.45
Start_time = 2:33.62    End_time = 2:54.45
Start time:  3:1.15
End time:  3:25.23
Start_time = 3:1.15    End_time = 3:25.23
Start time:  3:28.57
End time:  3:54.15
Start_time = 3:28.57    End_time = 3:54.15
Start time:  4:9.48
End time:  4:33.13
Start_time = 4:9.48    End_time = 4:33.13
Start time:  4:42.47
End time:  5:16.13
Start_time = 4:42.47    End_time = 5:16.13
Start time:  5:25.47
End

Moviepy - Building video 22_01_12_02_full_test.mp4.


MoviePy - Writing audio in 22_01_12_02_full_testTEMP_MPY_wvf_snd.mp3


chunk:  99%|█████████▊| 5510/5587 [00:17<00:00, 683.70it/s, now=None]


MoviePy - Done.


Moviepy - Writing video 22_01_12_02_full_test.mp4



t: 100%|██████████| 15186/15186 [20:46<00:00, 11.25it/s, now=None]


Moviepy - Done !


Moviepy - video ready 22_01_12_02_full_test.mp4


chunk:  13%|█▎        | 6740/50648 [35:42<01:42, 428.65it/s, now=None]

In [47]:
# my_edit.frame_count
import math
my_edit.list_start_frame
list_start_time = []
list_end_time = []

for time in my_edit.list_start_frame:
    list_start_time.append(time/my_edit.fps)
for time in my_edit.list_end_frame:
    list_end_time.append(time/my_edit.fps)

for item in range(len(list_start_time)):
    print('Start_time = '+str(math.floor(list_start_time[item]/60)) + ':' + str(round(((list_start_time[item]) % 60),2)) +'    End_time = ' +str(math.floor(list_end_time[item]/60)) + ':' + str(round(((list_end_time[item]) % 60),2)))

Start_time = 0:0.0    End_time = 0:4.5
Start_time = 0:23.83    End_time = 1:0.4
Start_time = 1:6.73    End_time = 1:12.23
Start_time = 1:13.78    End_time = 1:22.7
Start_time = 1:26.37    End_time = 1:32.33
Start_time = 1:39.67    End_time = 2:11.32
Start_time = 2:11.78    End_time = 2:15.28
Start_time = 2:33.62    End_time = 2:54.45
Start_time = 3:1.15    End_time = 3:25.23
Start_time = 3:28.57    End_time = 3:54.15
Start_time = 4:9.48    End_time = 4:33.13
Start_time = 4:42.47    End_time = 5:16.13
Start_time = 5:25.47    End_time = 5:54.42


In [6]:
# from PIL import Image
# from matplotlib import cm
# path = './Video_source/func_test_02_mix.mp4'
# vid = cv2.VideoCapture(path)
# t , pic = vid.read()
# # im = Image.fromarray(np.uint8(cm.gist_earth(pic)*255))
# # pic.shape
# imageRGB = cv2.cvtColor(pic, cv2.COLOR_BGR2RGB)
# im = Image.fromarray(imageRGB)
# # im = Image.fromarray(pic,mode="RGBa")
# im

In [16]:

for subclip in subclip_dict_list:
    subclip_source_video = source_video_clip
    print(subclip)
    start_time_0 = subclip["start time"] / my_edit.fps
    end_time_0 = subclip["end time"] / my_edit.fps
    print(str(start_time_0)+ str(end_time_0))
    current_subclip = subclip_source_video.subclip(start_time_0, end_time_0)
    print('done')
    subclip_list.append(current_subclip)
final_clip = concatenate_videoclips(subclip_list)

# source_video_clip.subclip(subclip["start time"], subclip["end time"])
final_clip.write_videofile(file_name)

{'start time': 0, 'end time': 270}
0.04.5
done
{'start time': 1430, 'end time': 3624}
23.83333333333333260.4
done
{'start time': 4004, 'end time': 4334}
66.7333333333333372.23333333333333
done
{'start time': 4427, 'end time': 4962}
73.7833333333333382.7
done
{'start time': 5182, 'end time': 5540}
86.3666666666666692.33333333333333
done
{'start time': 5980, 'end time': 7879}
99.66666666666667131.31666666666666
done
{'start time': 7907, 'end time': 8117}
131.78333333333333135.28333333333333
done
{'start time': 9217, 'end time': 10467}
153.61666666666667174.45
done
{'start time': 10869, 'end time': 12314}
181.15205.23333333333332
done
{'start time': 12514, 'end time': 14049}
208.56666666666666234.15
done
{'start time': 14969, 'end time': 16388}
249.48333333333332273.1333333333333
done
{'start time': 16948, 'end time': 18968}
282.46666666666664316.1333333333333
done
{'start time': 19528, 'end time': 21265}
325.46666666666664354.4166666666667
done
Moviepy - Building video 22_01_12_02_full_t

chunk:  13%|█▎        | 6729/50648 [00:24<01:42, 428.65it/s, now=None]

KeyboardInterrupt: 

chunk:  13%|█▎        | 6740/50648 [00:40<01:42, 428.65it/s, now=None]

In [8]:
subclip["start time"] / my_edit.fps

325.46666666666664

In [21]:
subclip_list

In [22]:
subclip_dict_list


[{'start time': 0.0, 'end time': 4.5},
 {'start time': 23.833333333333332, 'end time': 60.4},
 {'start time': 66.73333333333333, 'end time': 72.23333333333333},
 {'start time': 73.78333333333333, 'end time': 82.7},
 {'start time': 86.36666666666666, 'end time': 92.33333333333333},
 {'start time': 99.66666666666667, 'end time': 131.31666666666666},
 {'start time': 131.78333333333333, 'end time': 135.28333333333333},
 {'start time': 153.61666666666667, 'end time': 174.45},
 {'start time': 181.15, 'end time': 205.23333333333332},
 {'start time': 208.56666666666666, 'end time': 234.15},
 {'start time': 249.48333333333332, 'end time': 273.1333333333333},
 {'start time': 282.46666666666664, 'end time': 316.1333333333333},
 {'start time': 325.46666666666664, 'end time': 354.4166666666667}]